# Setup

In [2]:
%load_ext autoreload
%autoreload 2

import logging
import torch
import pytorch_lightning as pl
# import warnings

# warnings.filterwarnings('ignore')
# logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

## Pyspark Data Preproccessing

In [3]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T


data_path = 'data/'

spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.maxResultSize", "4g")
spark_conf.set("spark.executor.memory", "16g")
spark_conf.set("spark.executor.memoryOverhead", "4g")
spark_conf.set("spark.driver.memory", "16g")
spark_conf.set("spark.driver.memoryOverhead", "4g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

22/07/22 13:26:39 WARN Utils: Your hostname, vm2 resolves to a loopback address: 127.0.1.1; using 192.168.0.6 instead (on interface ens192)
22/07/22 13:26:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/22 13:26:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/22 13:26:40 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/07/22 13:26:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/22 13:26:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/07/22 13:26:41 WARN U

[('spark.driver.host', '192.168.0.6'),
 ('spark.driver.memoryOverhead', '4g'),
 ('spark.app.id', 'local-1658496401736'),
 ('spark.sql.warehouse.dir',
  'file:/home/kireev/pycharm-deploy/pytorch-lifestream/demo/spark-warehouse'),
 ('spark.local.dir', '../../spark_local_dir'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.executor.memory', '16g'),
 ('spark.app.startTime', '1658496400131'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '46639'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.shuffle.partitions', '200'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'PysparkDataPreprocessor'),
 ('spark.cores.max', '24'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.maxResultSize', '4g'),
 ('spark.executor.memoryOverhead', '4g')]

In [4]:
from urllib.request import urlretrieve
data, _ = urlretrieve(
    'https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true',
    'transactions_train.csv.gz')

source_data = spark.read.options(header=True, inferSchema=True).csv(data)
source_data.show(2)

+---------+----------+-----------+----------+
|client_id|trans_date|small_group|amount_rur|
+---------+----------+-----------+----------+
|    33172|         6|          4|    71.463|
|    33172|         6|         35|    45.017|
+---------+----------+-----------+----------+
only showing top 2 rows



In [5]:
from ptls.preprocessing import PysparkDataPreprocessor

preprocessor = PysparkDataPreprocessor(
    col_id='client_id',
    col_event_time='trans_date',
    event_time_transformation='none',
    cols_category=['small_group'],
    cols_numerical=['amount_rur'],
)

In [6]:
%%time

dataset_pysparkdf = preprocessor.fit_transform(source_data).persist()
dataset_pysparkdf.count()

22/07/22 13:27:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/22 13:27:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/22 13:27:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/22 13:27:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


CPU times: user 133 ms, sys: 31 ms, total: 164 ms
Wall time: 23.7 s


30000

In [7]:
dataset_pysparkdf.show()

+---------+--------------------+--------------------+--------------------+--------------------+
|client_id|          trans_date|         small_group|          amount_rur|          event_time|
+---------+--------------------+--------------------+--------------------+--------------------+
|      463|[1, 2, 5, 7, 8, 1...|[7, 1, 7, 3, 3, 2...|[62.14, 3.617, 43...|[1, 2, 5, 7, 8, 1...|
|      471|[0, 1, 1, 1, 1, 2...|[68, 1, 7, 4, 5, ...|[29.217, 135.852,...|[0, 1, 1, 1, 1, 2...|
|      496|[3, 3, 4, 5, 6, 8...|[1, 3, 1, 1, 1, 4...|[59.25, 32.057, 2...|[3, 3, 4, 5, 6, 8...|
|      833|[1, 1, 1, 2, 2, 3...|[1, 15, 17, 36, 4...|[45.413, 7.117, 4...|[1, 1, 1, 2, 2, 3...|
|     1238|[0, 3, 9, 10, 10,...|[3, 11, 3, 1, 3, ...|[4.579, 56.583, 8...|[0, 3, 9, 10, 10,...|
|     1342|[0, 4, 11, 11, 13...|[14, 3, 21, 3, 3,...|[3.435, 11.448, 7...|[0, 4, 11, 11, 13...|
|     1591|[2, 2, 3, 3, 17, ...|[20, 37, 6, 7, 1,...|[43.084, 91.544, ...|[2, 2, 3, 3, 17, ...|
|     1645|[2, 3, 3, 4, 4, 5...|[2, 1, 1

In [8]:
dataset_pysparkdf.dtypes

[('client_id', 'int'),
 ('trans_date', 'array<int>'),
 ('small_group', 'array<int>'),
 ('amount_rur', 'array<double>'),
 ('event_time', 'array<int>')]

In [9]:
test_df = dataset_pysparkdf.sample(fraction=0.2)
train_df = dataset_pysparkdf.subtract(test_df)

valid_df = train_df.sample(fraction=0.1)
train_df = train_df.subtract(valid_df)

print('Size of test dataset:', test_df.count())
print('Size of train dataset', train_df.count())
print('Size of valid dataset', valid_df.count())

test_df.write.parquet('test.parquet', mode='overwrite')
train_df.write.parquet('train.parquet', mode='overwrite')
valid_df.write.parquet('valid.parquet', mode='overwrite')

Size of test dataset: 6000


Size of train dataset 21699


Size of valid dataset 2301


## Data access 

In [1]:
from ptls.data_load.datasets import ParquetDataset, ParquetFiles

In [2]:
iterable_train = ParquetDataset(ParquetFiles('train.parquet'))

In [3]:
next(iter(iterable_train))

{'client_id': 20355,
 'trans_date': tensor([  0,   1,   2,  ..., 727, 727, 727], dtype=torch.int32),
 'small_group': tensor([ 4,  3,  6,  ...,  2,  3, 18], dtype=torch.int32),
 'amount_rur': tensor([16.3020,  6.6860,  6.6400,  ...,  0.0430, 10.7820,  3.6660],
        dtype=torch.float64),
 'event_time': tensor([  0,   1,   2,  ..., 727, 727, 727], dtype=torch.int32)}

In [4]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter, FeatureFilter

map_processed_train = MemoryMapDataset(
    data=iterable_train,
    i_filters=[
        SeqLenFilter(min_seq_len=25),
    ],
)

In [5]:
map_processed_train[0]

{'client_id': 20355,
 'trans_date': tensor([  0,   1,   2,  ..., 727, 727, 727], dtype=torch.int32),
 'small_group': tensor([ 4,  3,  6,  ...,  2,  3, 18], dtype=torch.int32),
 'amount_rur': tensor([16.3020,  6.6860,  6.6400,  ...,  0.0430, 10.7820,  3.6660],
        dtype=torch.float64),
 'event_time': tensor([  0,   1,   2,  ..., 727, 727, 727], dtype=torch.int32)}

**Attention!**

You cannot use a pretrained `coles-emb.pt` with a different preprocessor than the one used to train the model.
This is because preprocessor save specific category to embedding_id mapping during fit procedure.
Model use this mapping during pretrain.
Using different mapping at inference will corrupt output.